In [311]:
######################################################################################################################
# Assignment 2
# Subject - CSE535 Mobile Computing
# Author - Abhik Dey (1216907406), Sethu Manickam (1218452066), Rohith Eppepalli (1215350630), Sree Vashini Ravichandran (1217841794)
# Goal is:
#    1) Train Model to predict gestures - buy (1), communicate (2), fun (3), hope (4), mother (5), really (6)
#    2) Generate pickle file of the models
#
######################################################################################################################

import numpy as np
import pandas as pd
import scipy.fftpack as fftpack
import os
from sklearn.preprocessing import StandardScaler
import scipy.signal as signal
from sklearn.model_selection import KFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import Perceptron
from scipy.stats import kurtosis
import pickle

In [312]:
def normalizeData(data_frame):

    extracols = data_frame[['nose_x','nose_y','rightEye_x', 'rightEye_y','leftEye_x', 'leftEye_y']]
    data_frame.drop(['nose_x', 'nose_y', 'leftEye_x', 'leftEye_y', 'rightEye_x', 'rightEye_y'], axis=1, inplace=True)
    
    normalizedData = []
    df_cols = []
    fm = None
    for cols in data_frame.columns:
        df_cols.append(cols)
        col = data_frame[cols] # extracting each column
        mean = np.mean(col)
#         normData = (col - mean)/(np.max(col-mean) - np.min(col-mean))

        if cols.startswith("right") and cols.endswith("x"):
            normData = (col - extracols['nose_x'] / abs(extracols['leftEye_x'] - extracols['rightEye_x']))
        if cols.startswith("left") and cols.endswith("x"):
            normData = (col - extracols['nose_x'] / abs(extracols['leftEye_x'] - extracols['rightEye_x']))
        if cols.startswith("right") and cols.endswith("y"):
            normData = (col - extracols['nose_y'] / abs(extracols['leftEye_y'] - extracols['rightEye_y']))
        else:
            normData = (col - extracols['nose_y'] / abs(extracols['leftEye_y'] - extracols['rightEye_y']))
             
        df_norm_data = pd.DataFrame(normData, columns = [cols])
        
        rollMean = df_norm_data.rolling(window=10,min_periods=1).mean()
        fm = pd.concat([fm, rollMean], axis = 1, sort = False)
    return fm;


In [313]:
#code for feature mean
def calMean(data_frame):
    #cols = data_frame.columns
    
    df_cols = []
    mean_array = []


    feature_matrix = pd.DataFrame() #return feature matrix
    
    for cols in data_frame.columns:
        df_cols.append('Mean'+cols)
        meanval  = np.mean(data_frame[cols])
        mean_array.append(meanval);
        
    feature_matrix = pd.DataFrame([mean_array],columns = df_cols)
    
    return feature_matrix

In [314]:
#code for feature variance
def calVariance(data_frame):
    #cols = data_frame.columns
    
    df_cols = []
    variance_array = []


    feature_matrix = pd.DataFrame() #return feature matrix
    
    for cols in data_frame.columns:
        df_cols.append('Variance'+cols)
        varval  = np.var(data_frame[cols])
        variance_array.append(varval);
        
    feature_matrix = pd.DataFrame([variance_array],columns = df_cols)
    
    return feature_matrix

In [315]:
def calFFT(data_frame):
    #cols = data_frame.columns
    df_cols = []
    fft_val = []


    feature_matrix = pd.DataFrame() #return feature matrix
    
    for cols in data_frame.columns:
        df_cols.append('FFT_Peak1_'+cols)
        fft_values  = abs(fftpack.fft(data_frame[cols].values))
        val = set(fft_values)
        val = sorted(val, reverse = True)
        firstHighPeak = list(val)[1]
        fft_val.append(firstHighPeak)
        
    #return fft_val, df_cols
    feature_matrix = pd.DataFrame([fft_val],columns = df_cols)
    #print (feature_matrix)
    
    return feature_matrix

In [316]:
def calWelch(data_frame):
    df_cols = []
    welch_val = []


    feature_matrix = pd.DataFrame() 
    for cols in data_frame.columns:
        df_cols.append('maxAmplitude'+cols)
        v, welch_values  = np.array((signal.welch(data_frame[cols].values)))
        welch_val.append(np.sqrt(max(welch_values)))
    feature_matrix = pd.DataFrame([welch_val],columns = df_cols)
        
    return feature_matrix

In [317]:
def calMin(data_frame):
    
    df_cols = []
    min_array = []


    feature_matrix = pd.DataFrame() #return feature matrix
    
    for cols in data_frame.columns:
        df_cols.append('Min'+cols)
        minval  = min(data_frame[cols])
        min_array.append(minval);
        
    feature_matrix = pd.DataFrame([min_array],columns = df_cols)
    
    return feature_matrix

In [318]:
def calMax(data_frame):
    #cols = data_frame.columns
    
    df_cols = []
    max_array = []


    feature_matrix = pd.DataFrame() #return feature matrix
    
    for cols in data_frame.columns:
        df_cols.append('Max'+cols)
        maxval  = max(data_frame[cols])
        max_array.append(maxval);
        
    feature_matrix = pd.DataFrame([max_array],columns = df_cols)
    
    return feature_matrix

In [319]:
def calSlope(data_frame):
    #cols = data_frame.columns
    
    df_cols = []
    zc_array = []


    feature_matrix = pd.DataFrame() #return feature matrix
    
    for cols in data_frame.columns:
        df_cols.append('Slope1'+cols)
        df_cols.append('Slope2'+cols)

        threshold = np.mean(data_frame[cols])
        posSlope = 0
        posCount = 0
        negSlope = 0
        negCount = 0
        for i in range(len(data_frame[cols]) - 1):
            if (data_frame[cols][i] > data_frame[cols][i+1]):
                posSlope += data_frame[cols][i] - data_frame[cols][i+1]
                posCount += 1
            elif (data_frame[cols][i] < data_frame[cols][i+1]):
                negSlope += data_frame[cols][i+1] - data_frame[cols][i]
                negCount += 1
        zc_array.append(posSlope/posCount)
        zc_array.append(negSlope/negCount)
        
    feature_matrix = pd.DataFrame([zc_array],columns = df_cols)
    
    return feature_matrix

In [320]:
def execute_classifiers(model, X_train, X_test, Y_train, Y_test):
    model.fit(X_train,Y_train)
    score = model.score(X_test, Y_test)
    return score, model

In [ ]:
if __name__ == '__main__':

    gesture_dictionary = {
                           'hope' : ['leftElbow_x','leftElbow_y','rightElbow_x','rightElbow_y','leftWrist_y', 'rightWrist_y','leftWrist_x', 'rightWrist_x', 'rightEye_x', 'rightEye_y', 'leftEye_x', 'leftEye_y', 'nose_x', 'nose_y'],
                            'fun' : ['leftElbow_x','leftElbow_y','rightElbow_x','rightElbow_y','leftWrist_y', 'rightWrist_y','leftWrist_x', 'rightWrist_x', 'rightEye_x', 'rightEye_y', 'leftEye_x', 'leftEye_y', 'nose_x', 'nose_y'],
                            'really' : ['leftElbow_x','leftElbow_y','rightElbow_x','rightElbow_y','leftWrist_y', 'rightWrist_y','leftWrist_x', 'rightWrist_x', 'rightEye_x', 'rightEye_y', 'leftEye_x', 'leftEye_y', 'nose_x', 'nose_y'],
                            'buy' : ['leftElbow_x','leftElbow_y','rightElbow_x','rightElbow_y','leftWrist_y', 'rightWrist_y','leftWrist_x', 'rightWrist_x', 'rightEye_x', 'rightEye_y', 'leftEye_x', 'leftEye_y', 'nose_x', 'nose_y'],
                            'communicate' : ['leftElbow_x','leftElbow_y','rightElbow_x','rightElbow_y','leftWrist_y', 'rightWrist_y','leftWrist_x', 'rightWrist_x', 'rightEye_x', 'rightEye_y', 'leftEye_x', 'leftEye_y', 'nose_x', 'nose_y'],
                            'mother' : ['leftElbow_x','leftElbow_y','rightElbow_x','rightElbow_y','leftWrist_y', 'rightWrist_y','leftWrist_x', 'rightWrist_x', 'rightEye_x', 'rightEye_y', 'leftEye_x', 'leftEye_y', 'nose_x', 'nose_y']
                         } 
    
    gesture_labels = { 'buy' : 1,
                       'communicate' : 2,
                       'fun' : 3,
                       'hope': 4,
                       'mother': 5,
                       'really' : 6
                     }
    
    buy_feature_matrix = pd.DataFrame()
    communicate_feature_matrix = pd.DataFrame()
    fun_feature_matrix = pd.DataFrame()
    hope_feature_matrix = pd.DataFrame()
    mother_feature_matrix = pd.DataFrame()
    really_feature_matrix = pd.DataFrame()
    
    #Read the csv for Buy gesture -
    for ges in gesture_dictionary:
        feature_matrix = pd.DataFrame();
        for i in os.listdir('Datasets/'):
            fm = None
            if i.startswith(ges):
                #print ('i - ',i)
                #print ('ges - ',ges)
                keypoints = pd.read_csv('Datasets/'+i)
                temp_key_points = keypoints[gesture_dictionary[ges]];
                unique_key_points = normalizeData(temp_key_points)
                
                #Feature 1 -- For feature extraction of FFT
                resFFT = calFFT(unique_key_points)
                fm = pd.concat([fm,resFFT], axis = 1, sort = False)
                
                #Feature 2 -- For feature extraction of Welch
                resWelch = calWelch(unique_key_points)
                fm = pd.concat([fm,resWelch], axis = 1, sort = False)
                
                #Feature 3 -- For feature extraction of Mean
                resMean = calMean(unique_key_points)    
                fm = pd.concat([fm,resMean], axis = 1, sort = False)
                
                #Feature 4 -- For feature extraction of Min
                resMin = calMin(unique_key_points)
                fm = pd.concat([fm,resMin], axis = 1, sort = False)
                
                #Feature 5 -- For feature extraction of Max
                resMax = calMax(unique_key_points)
                fm = pd.concat([fm,resMax], axis = 1, sort = False)
                
                #Feature 6 -- For feature extraction of Variance
                resVariance = calVariance(unique_key_points)
                fm = pd.concat([fm,resVariance], axis = 1, sort = False)
                
                #Feature 7 -- For feature extraction of Slope
                resSlope = calSlope(unique_key_points)
                fm = pd.concat([fm,resSlope], axis = 1, sort = False)                
                
                
            feature_matrix = feature_matrix.append(fm, ignore_index = True, sort = False)
            
        if ges == 'buy':
            buy_feature_matrix = feature_matrix
            buy_feature_matrix['class'] = gesture_labels[ges]
        elif ges == 'communicate':
            communicate_feature_matrix = feature_matrix
            communicate_feature_matrix['class'] = gesture_labels[ges]
        elif ges == 'fun':
            fun_feature_matrix = feature_matrix
            fun_feature_matrix['class'] = gesture_labels[ges]
        elif ges == 'hope':
            hope_feature_matrix = feature_matrix
            hope_feature_matrix['class'] = gesture_labels[ges]
        elif ges == 'mother':
            mother_feature_matrix = feature_matrix
            mother_feature_matrix['class'] = gesture_labels[ges]
        elif ges == 'really':
            really_feature_matrix = feature_matrix
            really_feature_matrix['class'] = gesture_labels[ges]
            
    #        
    
    # Apply KFold to split Train and Test data and pass it to Models:
    final_feature_matrix = pd.concat([really_feature_matrix, hope_feature_matrix, fun_feature_matrix, 
                                     communicate_feature_matrix, mother_feature_matrix, buy_feature_matrix])[:]
    
    class_labels = pd.DataFrame({'class':final_feature_matrix['class'].tolist()})
    
    final_feature_matrix.drop(['class'], axis = 1, inplace = True)
    
    
    scaler = StandardScaler()
    final_feature_matrix_scaled = scaler.fit_transform(final_feature_matrix)
    
    #Perform Classifications
    
    ldascore = []
    ldamodel = []
    
#     knn1score = []
#     knn1model = []
    
    lrmodel = []
    lrscore = []
    
    svmscore = []
    svmmodel = []
    
    mlpscore = []
    mlpmodel = []
    
    kf = KFold(n_splits = 4,random_state=42, shuffle=True)
    
    for train_index, test_index in kf.split(final_feature_matrix_scaled):
        x_train, x_test, y_train, y_test = final_feature_matrix_scaled[train_index],final_feature_matrix_scaled[test_index],\
                                            class_labels['class'].iloc[train_index],class_labels['class'].iloc[test_index]
              
        #MLP Classifier   
        mlp_score, mlp_model = execute_classifiers(MLPClassifier(hidden_layer_sizes=(300,100,100), max_iter=5000,
                                                                 activation = 'relu', solver='adam',random_state=1),
                                                   x_train, x_test, y_train, y_test)
        mlpscore.append(mlp_score)
        mlpmodel.append(mlp_model)
        
        #Linear Discriminant Analysis
        lda_score, lda_model = execute_classifiers( LinearDiscriminantAnalysis(),x_train, x_test, y_train, y_test)
        ldascore.append(lda_score)
        ldamodel.append(lda_model)
        
#         # K Nearest Neighbours = 1
#         knn1_score, knn1_model = execute_classifiers( KNeighborsClassifier(n_neighbors = 1),x_train, x_test, y_train, y_test)
#         knn1score.append(knn1_score)
#         knn1model.append(knn1_model)

        # K Nearest Neighbours = 1
        lr_score, lr_model = execute_classifiers( LogisticRegression(random_state=0),x_train, x_test, y_train, y_test)
        lrscore.append(lr_score)
        lrmodel.append(lr_model)
        
        #SVM Classifier
        svm_score, svm_model = execute_classifiers(LinearSVC(random_state=0, tol=1e-5),x_train, x_test, y_train, y_test)
        svmscore.append(svm_score)
        svmmodel.append(svm_model)
        
        
        
    print ('LDA - ', ldascore)
    print ('LR - ',lrscore)
    print ('MLP - ', mlpscore)
    print ('SVM - ', svmscore)    

    with open('models/MLPModel.pkl', 'wb') as mlppklfile:
        pickle.dump(mlp_model, mlppklfile)
    mlppklfile.close()
    
    with open('models/LDAModel.pkl', 'wb') as ldapklfile:
        pickle.dump(lda_model, ldapklfile)
    ldapklfile.close()
    
    with open('models/LRModel.pkl', 'wb') as lrpklfile:
        pickle.dump(lr_model, lrpklfile)
    lrpklfile.close()
    
    with open('models/SVMModel.pkl', 'wb') as svmpklfile:
        pickle.dump(svm_model, svmpklfile)
    svmpklfile.close()

C:\Users\abhik\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\abhik\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1969: UserWarning: nperseg = 256 is greater than input length  = 117, using nperseg = 117
  .format(nperseg, input_length))
C:\Users\abhik\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1969: UserWarning: nperseg = 256 is greater than input length  = 132, using nperseg = 132
  .format(nperseg, input_length))
C:\Users\abhik\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1969: UserWarning: nperseg = 256 is greater than input length  = 144, using nperseg = 144
  .format(nperseg, input_length))
C:\Users\abhik\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1969: UserWarning: nperseg = 256 is gre

In [322]:
predict = lr_model.predict(x_test)

In [323]:
predict

array([6, 6, 5, 6, 4, 5, 3, 4, 6, 2, 6, 5, 6, 6, 6, 6, 5, 6, 4, 6, 4, 4,
       4, 4, 4, 2, 2, 4, 4, 4, 4, 4, 6, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5,
       3, 3, 5, 6, 3, 1, 1, 6, 1, 1, 4, 3, 4, 6, 6, 2, 4, 4, 4, 2, 2, 6,
       2, 1, 3, 2, 5, 5, 3, 5, 6, 3, 5, 5, 6, 5, 3, 5, 5, 5, 5, 1, 1, 1,
       3, 4, 3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [324]:
!python --version

Python 3.7.4


In [325]:
!pip freeze

absl-py==0.9.0
alabaster==0.7.12
anaconda-client==1.7.2
anaconda-navigator==1.9.7
anaconda-project==0.8.3
asn1crypto==1.0.1
astor==0.7.1
astroid==2.3.1
astropy==3.2.1
atomicwrites==1.3.0
attrs==19.2.0
Babel==2.7.0
backcall==0.1.0
backports.functools-lru-cache==1.5
backports.os==0.1.1
backports.shutil-get-terminal-size==1.0.0
backports.tempfile==1.0
backports.weakref==1.0.post1
beautifulsoup4==4.8.0
bitarray==1.0.1
bkcharts==0.2
bleach==3.1.0
blinker==1.4
bokeh==1.3.4
boto==2.49.0
boto3==1.12.36
botocore==1.15.36
Bottleneck==1.2.1
cachetools==3.1.1
certifi==2019.9.11
cffi==1.12.3
chardet==3.0.4
Click==7.0
cloudpickle==1.2.2
clyent==1.2.2
colorama==0.4.1
comtypes==1.1.7
conda==4.8.3
conda-build==3.18.9
conda-package-handling==1.6.0
conda-verify==3.4.2
contextlib2==0.6.0
cryptography==2.7
cycler==0.10.0
Cython==0.29.13
cytoolz==0.10.0
dask==2.5.2
decorator==4.4.0
defusedxml==0.6.0
distributed==2.5.2
docutils==0.15.2
entrypoints==0.3
et-xmlfile==1.0.1
fastcache==1.1.0
filelock==3.0.12
Flas